# Build a Simple LLM Application with LCEL

This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

In [1]:
! pip install langchain --quiet
! pip install python-dotenv --quiet
! pip install -qU langchain-openai --quiet

Use **LangSmith** to understand what happen in your LLM application.  You need to have a LangSmith account to use this feature. You can sign up for a free account at [LangSmith](https://smith.langchain.com/).

In [7]:
import os
from dotenv import load_dotenv

# Set environment variables
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
os.environ["AZURE_OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")

Create the model you will use, you can swap easily

In [8]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

Translate English to French using the model

In [14]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into French from Quebec!"),
    HumanMessage(content="hi!, I am looking for this amazing food from Montreal called poutine."),
]

# Since the tracer of langsmith is enabled you will see lot of information
model.invoke(messages)

Create output parser to work with the answer

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)

# Here this will remove the noise and only get the prompt answer
parser.invoke(result)

'Salut! Je cherche cette incroyable nourriture de Montréal appelée poutine.'

We can chain the model with this output parser. This means this output parser will get called every time in this chain.

In [17]:
chain = model | parser

chain.invoke(messages)

'Salut! Je suis à la recherche de cette délicieuse spécialité de Montréal appelée poutine.'

## Working with template

Now let's work with template to make our code better

In [21]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "spanish", "text": "hi, I am looking to the best restaurant in montreal"})

#result

result.to_messages()

[SystemMessage(content='Translate the following into aramaic:'),
 HumanMessage(content='hi, I am looking to the best restaurant in montreal')]

In [24]:
chain = prompt_template | model | parser

chain.invoke({"language": "spanish", "text": "hi, I am looking to the best restaurant in montreal"})

'Hola, estoy buscando el mejor restaurante en Montreal.'